In [1]:
from pydataset import data
from scipy import stats
import env
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import acquire
import prepare
import warnings

In [2]:
untidy = pd.read_sql("select * from attendance", acquire.get_connection('tidy_data'))

In [42]:
untidy

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [43]:
melted_attendence = pd.melt(untidy, id_vars = ["Unnamed: 0"])

In [44]:
melted_attendence

,Unnamed: 0,variable,value
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [45]:
melted_attendence['value'].replace(['P', 'T', 'H', 'A'],
                        [1, .9, .5, 0], inplace=True)

In [46]:
melted_attendence

,Unnamed: 0,variable,value
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9
5,Jane,2018-01-02,1.0
6,Billy,2018-01-02,0.9
7,John,2018-01-02,0.9
8,Sally,2018-01-03,0.9
9,Jane,2018-01-03,0.9


In [47]:
#Billy's attendance
billys_attendance =\
melted_attendence[
                  melted_attendence[ 
                                    "Unnamed: 0"] == "Billy"]

In [48]:
#John's attendance
johns_attendance =\
melted_attendence[
                  melted_attendence[ 
                                    "Unnamed: 0"] == "John"]

In [49]:
#Jane's attendance
janes_attendance =\
melted_attendence[
                  melted_attendence[ 
                                    "Unnamed: 0"] == "Jane"]

In [50]:
sallys_attendance =\
melted_attendence[
                  melted_attendence[ 
                                    "Unnamed: 0"] == "Sally"]

In [51]:
billys_attendance

,Unnamed: 0,variable,value
2,Billy,2018-01-01,0.0
6,Billy,2018-01-02,0.9
10,Billy,2018-01-03,0.0
14,Billy,2018-01-04,0.0
18,Billy,2018-01-05,0.5
22,Billy,2018-01-06,0.9
26,Billy,2018-01-07,1.0
30,Billy,2018-01-08,0.9


In [52]:
sum(billys_attendance.value)/8

0.525

In [53]:
attendance = pd.DataFrame({'name' : ['Billy', 'Jane', 'John', 'Sally'],
             'attendance' : [sum(billys_attendance.value)/8,
                             sum(janes_attendance.value)/8,
                             sum(johns_attendance.value)/8,
                            sum(sallys_attendance.value)/8]})

In [54]:
attendance

,name,attendance
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625


Coffee Levels

Read the coffee_levels table.
Transform the data so that each carafe is in it's own column.
Is this the best shape for the data?

In [55]:
untidy_coffee = pd.read_sql("select * from coffee_levels", acquire.get_connection('tidy_data'))

In [56]:
untidy_coffee

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [63]:
untidy_coffee.pivot(index = 'hour', columns='coffee_carafe')

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

### I would say that yes, this is the best way to represent the data because it is organized and easy to read

Cake Recipes

Read the cake_recipes table. 

This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.


Tidy the data as necessary.


Which recipe, on average, is the best? recipe b
Which oven temperature, on average, produces the best results? 275
Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees